<a href="https://colab.research.google.com/github/rick-phyto/cheminformatics/blob/main/Jasial_80P40_CSVload_getindices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import auth #gsheets
#import gspread #gsheets
#from google.auth import default #gsheets
#autenticating to google #gsheets
#auth.authenticate_user() #gsheets
#creds, _ = default() #gsheets
#gc = gspread.authorize(creds) #gsheets

from google.colab import drive #new for csv
drive.mount('/content/gdrive') #new for csv


import pandas as pd

Mounted at /content/gdrive


In [ ]:
# Install RDKit.
%%capture
!pip install rdkit

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np

import requests
from bs4 import BeautifulSoup
import re

In [ ]:
#works
#something=pd.read_csv('gdrive/My Drive/Colab Notebooks/pubchemtest_for_csv.csv')
#something=pd.read_csv('gdrive/My Drive/Colab Notebooks/for gcolab with_percentiles_csv_437k_bajorath_greater_40C_only_392269.csv') #some NaNs in here
something=pd.read_csv('gdrive/My Drive/Colab Notebooks/Copy of bajorath 437000 80P_40C 345208.csv') #looks cleaner based on head-tail.
something.head
#worksheet = gc.open('onetwothree').sheet1
#worksheet


In [ ]:
something.shape

(345207, 11)

In [ ]:
something.head

<bound method NDFrame.head of              cid  #PrimaryAssays  AssayPromiscuity_Primary  APP %  \
0              6             292                        13  4.50%   
1             19             317                         0  0.00%   
2             40             190                         0  0.00%   
3             72             383                         5  1.30%   
4             86             345                        14  4.10%   
...          ...             ...                       ...    ...   
345202  56642970             430                         3  0.70%   
345203  56642971             313                         6  1.90%   
345204  56643184             282                         3  1.10%   
345205  56643185             283                         4  1.40%   
345206  56643186             277                         5  1.80%   

        TargetPromiscuity_Primary  TPP %  #ConfirmatoryAssays  \
0                               9  3.10%                   92   
1          

In [ ]:
#rows = worksheet.get_all_values()


In [ ]:
#something = pd.DataFrame.from_records(rows)
#something

In [ ]:
#something.loc[:,0]
#something

In [ ]:
#cids = something.loc[:,0].tolist()
cids = something['cid'].tolist() 
#cids[392240:392259] #that's rather convenient - the CIDs in the 80P-40C set top out at just under 57,000,000 :) Maybe an selection-result of what researchers assay with high prevalence. Even the 40C only list just has a handful of 73,000,000's at the very end
#df.to_csv('file.csv', index=False)
didthiswork = something.to_csv('gdrive/My Drive/Colab Notebooks/JasialToPubchem-indices.csv',columns=["cid"], index=False) 
#print(didthiswork) #seems to work even tho this line is useless

None


In [ ]:
#using full pubchem xml / beautifulsoup approach - works
for cid in cids:
  #a_get_string = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/" + cid + "/xml"
  a_get_string = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/" + str(cid) + "/xml" #works
  data = requests.get(a_get_string) #10621 hesperidin
  #data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/" + cid + "/xml") #10621 hesperidin
  #print(data)
  html = BeautifulSoup(data.content, "xml")
  html.find_all("PC-Urn_label")
  smiles_tag = html.find(name="PC-Urn_label", string="SMILES")
  smiles_parents = smiles_tag.find_parent("PC-InfoData")
  smiles = smiles_parents.find('PC-InfoData_value_sval').string
  #print(smiles)
  chemical_mol = Chem.MolFromSmiles(smiles)
  Number_of_phenols = Chem.Fragments.fr_phenol(chemical_mol)
  #Number_of_phenols_list[cid-1] = int(Chem.Fragments.fr_phenol(chemical_mol))
  #print(Number_of_phenols_list)
  print(Number_of_phenols)

In [ ]:
#using just pubchem smiles txt reponse query - works
for cid in cids:
  #a_get_string = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/" + cid + "/xml"
  #a_get_string = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/" + str(cid) + "/xml" #works
  a_get_string = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/" + str(cid) + "/property/CanonicalSmiles/txt"
  data = requests.get(a_get_string) #10621 hesperidin
  #data.content
  out_smiles_string_b = data.content[0:(-1+len(data.content))] #remove newline 
  #out_smiles_string_b
  out_smiles = out_smiles_string_b.decode('ASCII') #remove 'b'
  #out_smiles
  #data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/" + cid + "/xml") #10621 hesperidin
  #print(data)
  #html = BeautifulSoup(data.content, "xml")
  #html.find_all("PC-Urn_label")
  #smiles_tag = html.find(name="PC-Urn_label", string="SMILES")
  #smiles_parents = smiles_tag.find_parent("PC-InfoData")
  #smiles = smiles_parents.find('PC-InfoData_value_sval').string
  smiles = out_smiles
  #print(smiles)
  chemical_mol = Chem.MolFromSmiles(smiles)
  Number_of_phenols = Chem.Fragments.fr_phenol(chemical_mol)
  #Number_of_phenols_list[cid-1] = int(Chem.Fragments.fr_phenol(chemical_mol))
  #print(Number_of_phenols_list)
  print(Number_of_phenols)

3
3
12
11
5
3
0
3
25
2
2
2
2
1
2
6
5
2
5
2
4
0
0
9
3
5
0
2
4
1
8
3
1
8
3
3
8
2
2
3
0
2
4
2
0
15
8
2
15
3
2
2
3
1
0
0
3
0
3
0
2
5
3
3
4
6
0
2
4
4
3
3
1
0
1
5
2
4
3
3
4
0
0
0
2
2
9
0
1
4
3
3
5
2
2
4
4
4
1
4
4
3
4
0
3
3
4
4
3
2
2
3
2
3
4
2
3
0
2
0
2
3
2
0
3
2
5
3
0
2
4
0
10
2
3
4
0
0
1
0
0
2
0
2
4
2
1
2
1
0
0
2
2
1
1
3
2
2
4
0
2
3
2
2
1
1
0
2
4
1
0
0
1
2
2
1
0
0
2
2
2
4
0
2
2
1
2
2
0
2
2
2
0
0
0
0
2
2
2
1
3
0
3
0
2
1
2
0
2
0
0
2
0
2
0
2
0
2
0
0
0
2
0
0
0
0
0
0
0
1
0
1
0
1
0
0
1
0
1
1
0
1
0
3
0
0
2
0
1
0
0
0
0
0
1
0
0
0
0
0
0
3
3
3
4
2
4
1
2
2
2
0


In [ ]:
#quickcheck - deleteme 
cyclohexanol_smiles = 'C1=CC=C(C=C1)O' #phenol # 'C1CCC(CC1)O' #cyclohexanol
cyclohexanol_mol = Chem.MolFromSmiles(cyclohexanol_smiles)
Number_of_phenols_ch = Chem.Fragments.fr_phenol(cyclohexanol_mol)
print(Number_of_phenols_ch)

1
